In [1]:
import tensorflow as tf
import numpy as np
from hourglass_tiny import HourglassModel
import configparser
import os
import pandas as pd
from keras.preprocessing import image
import scipy.io
from scipy.misc import imresize
import matplotlib.pyplot as plt

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


## process the data

In [2]:
# get the path of the data
data_path = os.path.join('Sitting','img')
label_path = os.path.join('Sitting','masks')
data_names = sorted(os.listdir(data_path))
label_names = sorted(os.listdir(label_path))
data_addr = []
for i in range(len(data_names)):
    data_addr.append(os.path.join('Sitting/img',data_names[i]))

label_addr = []
for i in range(len(label_names)):
    label_addr.append(os.path.join('Sitting/masks',label_names[i]))
The_Main_List = pd.DataFrame(data_names,columns = ['data_name'])
temp_frame1 = pd.DataFrame(label_names,columns=['label_name'])
temp_frame2 = pd.DataFrame(data_addr,columns=['data_addr'])
temp_frame3 = pd.DataFrame(label_addr,columns=['label_addr'])
The_Main_List = pd.concat([The_Main_List,temp_frame1],axis=1)
The_Main_List = pd.concat([The_Main_List,temp_frame2],axis=1)
The_Main_List = pd.concat([The_Main_List,temp_frame3],axis=1)
The_Main_List.head()

,data_name,label_name,data_addr,label_addr
0,image0.jpg,image0.mat,Sitting/img\image0.jpg,Sitting/masks\image0.mat
1,image1.jpg,image1.mat,Sitting/img\image1.jpg,Sitting/masks\image1.mat
2,image10.jpg,image10.mat,Sitting/img\image10.jpg,Sitting/masks\image10.mat
3,image100.jpg,image100.mat,Sitting/img\image100.jpg,Sitting/masks\image100.mat
4,image101.jpg,image101.mat,Sitting/img\image101.jpg,Sitting/masks\image101.mat


In [3]:
# load one image
def load_image(path):
    img = image.load_img(path,target_size=(256,256))
    img = image.img_to_array(img)
    return img
# laod one label as mask
def load_mask(path):
    label = scipy.io.loadmat(path)
    label = label['M']
    label_resize = imresize(label,[64,64],interp='nearest')
    label_resize = label_resize.astype('float32')
    label_floor = np.floor(label_resize/18)
    return label_floor

In [5]:
# load all data
data = np.zeros([6,256,256,3])
for i,path in enumerate(The_Main_List.data_addr):
    data[i] = load_image(path)
    if i == 5:
        break

# load all labels
label = np.zeros([6,64,64])
for i,path in enumerate(The_Main_List.label_addr):
    label[i] = load_mask(path)
    if i == 5:
            break

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # Remove the CWD from sys.path while we load stuff.


In [6]:
# split train and test
np.random.seed(seed=1993)
rnd = np.arange(len(data))
# train_idx = rnd < 180
# valid_idx = rnd >= 180
train_data = data
#test_data = data[valid_idx]
train_label = label
#test_label = label[valid_idx]

In [7]:
def augmentor_with_keras(img_asarr,mask):
    temp = np.zeros([256,256,4])
    temp[:,:,0:3] = img_asarr
    temp[128-32:128+32,128-32:128+32,3]= mask
    after_rot = image.random_rotation(temp,rg=20,
                                      row_axis=0,col_axis=1,channel_axis=2,fill_mode='constant')
    after_zoom = image.random_zoom(after_rot,zoom_range=[0.7,1.3],
                                   row_axis=0,col_axis=1,channel_axis=2,fill_mode='constant')
    # random_shift
    return after_zoom[:,:,0:3],after_zoom[128-32:128+32,128-32:128+32,3]

In [8]:
def transfer_mask_to_gtmap(mask):
    map = np.zeros([64,64,15])
    for i in range(15):
        for n in range(64):
            for m in range(64):
                if mask[n][m] == i:
                    map[n,m,i] = 1.0
    # transform to gtMap
    gtmap = np.zeros([4,64,64,15])
    for i in range(4):
        gtmap[i] = map
    return gtmap

In [9]:
def get_next_batch(imgs,masks):
    length = len(imgs)
    idx = np.random.randint(0,length)
    img = imgs[idx]
    mask = masks[idx]
    return img,mask

In [10]:
def get_augmentor_batch(img,mask,aug_level):
    gtmap = transfer_mask_to_gtmap(mask)
    x_batch = np.zeros([aug_level,256,256,3])
    y_batch = np.zeros([aug_level,4,64,64,15])
    x_batch[0]=img
    y_batch[0]=gtmap
    for i in range(aug_level-1):
        aug_img,aug_mask = augmentor_with_keras(img,mask)
        gtmap = transfer_mask_to_gtmap(aug_mask)
        x_batch[i+1] = aug_img
        y_batch[i+1] = gtmap
    return x_batch,y_batch

In [11]:
def cal_acc(pred,mask):
    length = pred.shape[0]
    wideth = pred.shape[1]
    help_arr = np.zeros([length,wideth])
    for i in range(length):
        for j in range(wideth):
            if pred[i,j] == mask[i,j]:
                help_arr[i,j] = 1
    acc = np.mean(help_arr)
    return acc

In [12]:
def cal_iou(pred,mask):
    length = pred.shape[0]
    wideth = pred.shape[1]
    # find the pixel that at the same time in two map
    p_in_two = 0
    for i in range(length):
        for j in range(wideth):
            if (pred[i,j] != 0) and (mask[i,j] != 0):
                p_in_two = p_in_two + 1
    # find the pixel that not equal to zero
    p_all = 0
    for i in range(length):
        for j in range(wideth):
            if (pred[i,j] != 0) or (mask[i,j] != 0):
                p_all = p_all + 1
    if p_all != 0:
        iou = p_in_two/p_all
    else:
        iou = 0
    return iou

## build net 

In [13]:
def process_config(conf_file):
	"""
	"""
	params = {}
	config = configparser.ConfigParser()
	config.read(conf_file)
	for section in config.sections():
		if section == 'DataSetHG':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Network':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Train':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Validation':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Saver':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
	return params
print('--Parsing Config File')
params = process_config('config.cfg')

--Parsing Config File

In [14]:
model = HourglassModel(nFeat=params['nfeats'], nStack=params['nstacks'], 
                       nModules=params['nmodules'], 
                       nLow=params['nlow'], outputDim=params['num_joints'], 
                       batch_size=params['batch_size'], attention = params['mcam'],
                       training=True, drop_rate= params['dropout_rate'], 
                       lear_rate=params['learning_rate'], 
                       decay=params['learning_rate_decay'], 
                       decay_step=params['decay_step'], 
                       name=params['name'], 
                       logdir_train=params['log_dir_train'], 
                       logdir_test=params['log_dir_test'], 
                       tiny= params['tiny'], w_loss=params['weighted_loss'] , 
                       joints= params['joint_list'],modif=False)
model.generate_model()

CREATE MODEL:

---Inputs : Done (0 sec.)

---Graph : Done (12 sec.)

---Loss : Done (0 sec.)

In [15]:
# output: (None,8,64,64,15)
output = model.get_output()
loss = model.loss
out = output[0,7]
pred_seg = tf.argmax(out,axis=2)
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [16]:
saver = tf.train.Saver()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [17]:
# train
iteration = []
acc_list = []
iou_list = []
for i in range(10):
    x_b,y_b = get_next_batch(train_data,train_label)
    x,y = get_augmentor_batch(x_b,y_b,3)
    sess.run(train_step,feed_dict={model.img:x, model.gtMaps:y})

    x_for_eval = np.expand_dims(x_b,axis=0)
    pred = sess.run(pred_seg,feed_dict={model.img:x_for_eval})
    acc = cal_acc(pred,y_b)
    iou = cal_iou(pred,y_b)
    print('acc:%f',acc,'  ','iou:%f',iou)
    iteration.append(i)
    acc_list.append(acc)
    iou_list.append(iou)

acc:%f

0.01416015625

iou:%f

0.0853418593212002

acc:%f

0.01708984375

iou:%f

0.09448818897637795

KeyboardInterrupt: 

In [ ]:
# save iou and acc
eval_info = pd.DataFrame([iteration,acc_list,iou_list],columns=['iter','accracy','iou'])
eval_info.to_csv('training_info.csv')

In [ ]:
# save the network
path = "my_net"
is_exist = os.path.exists(path)
if not is_exist:
    os.makedirs(path)
full_path = os.path.join(path,'save_net.ckpt')
save_path = saver.save(sess,full_path)